## Data Analysis of Stuidents Performance

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
print(os.getcwd())

c:\Users\pakya\Desktop\Projects\Data Science\End_To_End_ML_Project\notebook


In [3]:
# read data
df = pd.read_csv('data\StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
df.columns
df.isna().value_counts()
df.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [5]:
df['gender'].value_counts()
# the data seems to be non biased based on gender

gender
female    518
male      482
Name: count, dtype: int64

In [6]:
categorical_features = df.select_dtypes(include='object').columns
print(categorical_features)
for col in df.columns:
    if col in categorical_features:
        print(f'The unique values in column {col} are: {df[col].unique()}')



Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')
The unique values in column gender are: ['female' 'male']
The unique values in column race/ethnicity are: ['group B' 'group C' 'group A' 'group D' 'group E']
The unique values in column parental level of education are: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
The unique values in column lunch are: ['standard' 'free/reduced']
The unique values in column test preparation course are: ['none' 'completed']


In [7]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [10]:
from sklearn.preprocessing import LabelEncoder
df['parental level of education'].value_counts()

le = LabelEncoder()
encoded = le.fit_transform(df['parental level of education'])
print(type(encoded))

<class 'numpy.ndarray'>


In [18]:
df['parental level of education'].shape
print(encoded.shape)
df['parental level of education'].value_counts()

(1000,)


parental level of education
some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: count, dtype: int64

In [23]:
le.transform(['some high school'])

array([5])

In [24]:
from sklearn.preprocessing import OneHotEncoder

# let's try one hot encoding the same column
df['parental level of education'].value_counts()

parental level of education
some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: count, dtype: int64

In [104]:
oh = OneHotEncoder()
df['parental level of education'].values
ohencode = oh.fit_transform(df['parental level of education'].to_numpy().reshape(-1,1))
print(type(ohencode))
ohencode.toarray().shape
ohencode.toarray()[0]

<class 'scipy.sparse._csr.csr_matrix'>


array([0., 1., 0., 0., 0., 0.])

In [42]:
cat_columns = list(df.select_dtypes(include='O').columns)

In [61]:
from sklearn.pipeline import Pipeline

cat_pipeline = Pipeline(
    steps=[('One_Hot_Encoding', OneHotEncoder())]
)

In [88]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

cat_pipeline = Pipeline(
    steps=[('simple_imputer', SimpleImputer(strategy='most_frequent')),
           ('one_hot_encoder', OneHotEncoder()),
           ('standard_scaler', StandardScaler(with_mean=False))]
)

In [55]:
cat_pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('One Hot Encoding', OneHotEncoder())])>

In [89]:
# the transform would return an array
from sklearn.compose import ColumnTransformer

coltrans = ColumnTransformer(transformers=[('onehot_encode', cat_pipeline, cat_columns)])

In [90]:
coltrans.get_params

<bound method ColumnTransformer.get_params of ColumnTransformer(transformers=[('onehot_encode',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder()),
                                                 ('standard_scaler',
                                                  StandardScaler(with_mean=False))]),
                                 ['gender', 'race/ethnicity',
                                  'parental level of education', 'lunch',
                                  'test preparation course'])])>

In [91]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [100]:
resarr = coltrans.fit_transform(df)
resarr.shape
print(type(resarr), resarr.shape)
resarr.toarray().shape
print(resarr.toarray()[0])
df.iloc[0,:]


<class 'scipy.sparse._csr.csr_matrix'> (1000, 17)
[2.00129726 0.         0.         2.54906371 0.         0.
 0.         0.         3.09973598 0.         0.         0.
 0.         0.         2.08980597 0.         2.08588794]


gender                                    female
race/ethnicity                           group B
parental level of education    bachelor's degree
lunch                                   standard
test preparation course                     none
math score                                    72
reading score                                 72
writing score                                 74
Name: 0, dtype: object

In [95]:
X = df.drop(columns=['math score'], axis=1)
Y = df['math score']

print(X.head())
print(Y.head())

   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  reading score  writing score  
0                    none             72             74  
1               completed             90             88  
2                    none             95             93  
3                    none             57             44  
4                    none             78             75  
0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64


In [96]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [97]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=50)
print(f'Total Records - : {df.shape}')
print(f'X Train Records - : {x_train.shape}')
print(f'X Test Records - : {x_test.shape}')
print(f'Y Train Records - : {y_train.shape}')
print(f'Y Test Records - : {y_test.shape}')

Total Records - : (1000, 8)
X Train Records - : (750, 7)
X Test Records - : (250, 7)
Y Train Records - : (750,)
Y Test Records - : (250,)


In [98]:
x_train.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
651,female,group A,high school,standard,completed,80,76
492,female,group C,some college,standard,none,83,90
210,male,group D,some high school,free/reduced,completed,79,79
220,female,group D,high school,standard,completed,68,74
458,female,group E,bachelor's degree,standard,none,100,100


In [105]:
train_arr = coltrans.fit_transform(x_train)

In [107]:
train_arr

<750x17 sparse matrix of type '<class 'numpy.float64'>'
	with 3750 stored elements in Compressed Sparse Row format>

In [110]:
print(train_arr.shape, y_train.shape)

(750, 17) (750,)


In [111]:
np.c_[train_arr.toarray(),np.array(y_train)]

array([[ 2.00445931,  0.        ,  3.4826884 , ...,  2.0974939 ,
         0.        , 68.        ],
       [ 2.00445931,  0.        ,  0.        , ...,  0.        ,
         2.0974939 , 83.        ],
       [ 0.        ,  2.00445931,  0.        , ...,  2.0974939 ,
         0.        , 80.        ],
       ...,
       [ 2.00445931,  0.        ,  0.        , ...,  2.0974939 ,
         0.        , 79.        ],
       [ 0.        ,  2.00445931,  0.        , ...,  2.0974939 ,
         0.        , 73.        ],
       [ 0.        ,  2.00445931,  3.4826884 , ...,  0.        ,
         2.0974939 , 53.        ]])

In [116]:
from scipy.sparse import csr_matrix
type(coltrans.transform(x_test))==csr_matrix

True